El objetivo de este cuaderno es complementar la sesión de bayesiana y asentar los conceptos más aplicables:

- Comparación de muestras: La comparación de muestras extrae las distribuciones que están debajo de dos o más juegos de datos y compara valores extraídos de estas distribuciones para determinar cuál es "mejor".
- Inferencia de los parámetros de una distribución: determina los componentes de la distribución de una serie de datos

In [1]:
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
import pymc3 as pm
import arviz as az
import numpy as np

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# Esto está para ayudaros, no para que lo escribáis

import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

matplotlib.rcParams['figure.figsize'] = [16,8]



colores = ['orange', 'lightblue', 'lightgreen', 'green', 'red']

def dibujar_muestras(traza, nombres, res=None, datos=None):
    plt.figure()
    for i,x in enumerate([x for x in nombres if x != 'delta']):
        print(f'dibujando histograma para {x}')
        sns.histplot(traza.posterior[x].to_series(), color=colores[i])
    
    if 'delta' in nombres:
        plt.figure()
        delta = traza.posterior['delta'].to_series()
        delta_pos = delta[delta>0]
        delta_neg = delta[delta<0]
        sns.histplot(delta_neg, color='red')
        sns.histplot(delta_pos, color='lightgreen')
    
    if res:
        plt.figure()
        sns.histplot(np.concatenate(res['obs_2'])[:len(datos)], color='lightgreen')
        sns.histplot(datos, color='orange')

### Comparación de muestras

**0 - Comparación de dos muestras uniforme**

La distribución uniforme discreta determina que todos los valores contenidos tienen la misma probabilidad de figurar. Ver: la posibilidad de sacar cualquier valor en un dado

In [ ]:
with pm.Model() as model:
    dist_1 = pm.Uniform('dist_1', lower=20, upper=30)
    dist_2 = pm.Uniform('dist_2', lower=30, upper=40)
    
    delta = pm.Deterministic('delta', dist_2 - dist_1)
    
    trace = pm.sample(draws = 10000, return_inferencedata=True)

    az.plot_posterior(trace, figsize=(16,8))

INFO (theano.gof.compilelock): Waiting for existing lock by unknown process (I am process '12592')
INFO (theano.gof.compilelock): To manually release the lock, delete C:\Users\aleex\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-Intel64_Family_6_Model_158_Stepping_10_GenuineIntel-3.8.5-64\lock_dir
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [dist_2, dist_1]


In [ ]:
dibujar_muestras(trace, ['dist_1','dist_2','delta'])

### Ejercicio 1 - Comparación de muestras uniformes

Partiendo del ejemplo de código anterior, obtén muestras para una distribución uniforme que vaya desde 10 hasta 15 y otra que vaya de 13 a 16.

In [ ]:
with pm.Model() as model:
    dist_1 = pm.Uniform('dist_1', lower=10, upper=15)
    dist_2 = pm.Uniform('dist_2', lower=13, upper=16)
    
    delta = pm.Deterministic('delta', dist_2 - dist_1)
    
    trace = pm.sample(draws = 10000, return_inferencedata=True)

    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['dist_1','dist_2','delta'])

In [ ]:
sns.histplot(trace.posterior['delta'].to_series(), color='green')

In [ ]:
mejor_2 = np.mean(trace['posterior']['delta'].to_series() > 0)
mejor_1 = np.mean(trace['posterior']['delta'].to_series() < 0)

print(f'Dist1 es mejor en el {int(mejor_1*100)} % de los casos, Dist2 es mejor en el {int(mejor_2*100)} % de los casos')

### Ejercicio 2 - Compara una muestra normal con una muestra uniforme

Se pretende que la muestra normal sea muy superior a la uniforme, en ello nos basamos para numerar los parámetros

In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

with pm.Model() as model:
    dist_1 = pm.Uniform('dist_1', 1, 6) # minimo y maximo
    dist_2 = pm.Normal('dist_2', 5, 2) # media y desviacion tipica
    
    delta = pm.Deterministic('delta', dist_2 - dist_1)
    
    trace = pm.sample(return_inferencedata=True)

    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['dist_1', 'dist_2', 'delta'])

In [ ]:
mejor_2 = np.mean(trace['posterior']['delta'].to_series() > 0)
mejor_1 = np.mean(trace['posterior']['delta'].to_series() < 0)

print('Dist1 es mejor en el {:.2f} % de los casos, Dist2 es mejor en el {:.2f} % de los casos'.format(mejor_1*100,mejor_2*100))

### Ejercicio 3 - Distribución de Poisson vs uniforme

Cambia el ejercicio anterior para que, en lugar de utilizar una distribución normal utilice una de poisson.

Ojo!, la distribución de Poisson acepta un único parámetro

¿Has tenido que hacer transformaciones en los datos para llegar a alguna conclusión?

In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

with pm.Model() as model:
    dist_1 = pm.Uniform('dist_1', lower=5, upper=15)
    dist_2 = pm.Poisson('dist_2',20)
    
    delta = pm.Deterministic('delta', dist_2 - dist_1)
    
    trace = pm.sample(return_inferencedata=True)

    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['dist_1', 'dist_2', 'delta'])

In [ ]:
mejor_2 = np.mean(trace['posterior']['delta'].to_series() > 0)
mejor_1 = np.mean(trace['posterior']['delta'].to_series() < 0)

print('Dist1 es mejor en el {:.2f} % de los casos\nDist2 es mejor en el {:.2f} % de los casos'.format(mejor_1*100,mejor_2*100))

## Detectar parámetros de una distribución

Para estos ejercicios partiremos de distribuciones que iremos generando a mano y trataremos de comprobar si el ajuste que hace PyMC3 es bueno.

### 0 - Obtener parámetros de una distribución uniforme

In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

# Obtenemos 1000 medidas de una distribución uniforme entre 0 y 10
datos = np.random.uniform(10, 20, 1000)

with pm.Model() as model:
    
    p_A = pm.Uniform('p_A', lower=5, upper=15)
    p_B = pm.Uniform('p_B', lower=15, upper=25)
    
    obs = pm.Uniform('obs', lower=p_A, upper=p_B, observed=datos)
    
    trace = pm.sample(return_inferencedata=True)

    res = pm.sample_posterior_predictive(trace, samples=1000)
        
    az.plot_posterior(trace, figsize=(16,8))
    
#dibujar_muestras(trace, ['p_A', 'p_B'], res, datos)

In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

# Obtenemos 1000 medidas de una distribución uniforme entre 0 y 10
datos = np.random.uniform(10, 20, 1000)


with pm.Model() as model:
    # Como se trata de ajustar a una normal vamos a probar con media entre 10 y 20  y desviación entre 0 y 5
    p_A = pm.Uniform('p_A', lower=10, upper=20)
    p_B = pm.Uniform('p_B', lower=0, upper=5)
    
    obs = pm.Normal('obs', p_A, p_B, observed=datos)
    
    trace = pm.sample(return_inferencedata=True)

    res = pm.sample_posterior_predictive(trace, samples=1000)
        
    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['p_A', 'p_B'], res, datos)